In [1]:
import pandas as pd
import gdown
import os

### Importing and cleaning BDL data

In [4]:
# Importing BDL data.

cena_mieszkan = pd.DataFrame()
mediana_wieku = pd.DataFrame()
saldo_migracji = pd.DataFrame() # dotyczy osób w wieku produkcyjnym
wsp_feminizacji = pd.DataFrame()
wydatki_powiat = pd.DataFrame() # w złotówkach
wynagrodzenia = pd.DataFrame() # w złotówkach

bdl_dfs_list = [
    cena_mieszkan,
    mediana_wieku,
    saldo_migracji,
    wsp_feminizacji,
    wydatki_powiat,
    wynagrodzenia
]

file_ids = {
    "cena_mieszkan": "12UWwTtqRazIEctdzxib34fs8I_-ZUOD7",
    "mediana_wieku": "1D8jvho56cqn5dNGhMPaHujF3tmoQ1L6K",
    "saldo_migracji": "1sVL3iXGzc9D2k38ogErl84mVIzl6SzIP",
    "wsp_feminizacji": "163IRO4q5iU-MdyN3kYXkFHDavpyr353f",
    "wydatki_powiat": "1YMXflzDmozNqxQLpcd6oc1e1vtUVyUtS",
    "wynagrodzenia": "104z3VjCcv66e2L034D-o3vJY2wROJwCY"
}

for i in range(len(file_ids)):
    url = f"https://drive.google.com/uc?id={list(file_ids.values())[i]}"
    file_extension = "downloaded_file.csv"
    gdown.download(url, file_extension, quiet=False)
    bdl_dfs_list[i] = pd.read_csv(file_extension, sep=";", on_bad_lines='skip')

Downloading...
From: https://drive.google.com/uc?id=12UWwTtqRazIEctdzxib34fs8I_-ZUOD7
To: C:\Users\Konrad\Desktop\Magisterka\Kod\2024-mgr-sluzba-cywilna\Python\Code\downloaded_file.csv
100%|██████████| 204k/204k [00:00<00:00, 12.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1D8jvho56cqn5dNGhMPaHujF3tmoQ1L6K
To: C:\Users\Konrad\Desktop\Magisterka\Kod\2024-mgr-sluzba-cywilna\Python\Code\downloaded_file.csv
100%|██████████| 144k/144k [00:00<00:00, 3.03MB/s]
Downloading...
From: https://drive.google.com/uc?id=1sVL3iXGzc9D2k38ogErl84mVIzl6SzIP
To: C:\Users\Konrad\Desktop\Magisterka\Kod\2024-mgr-sluzba-cywilna\Python\Code\downloaded_file.csv
100%|██████████| 259k/259k [00:00<00:00, 3.68MB/s]
Downloading...
From: https://drive.google.com/uc?id=163IRO4q5iU-MdyN3kYXkFHDavpyr353f
To: C:\Users\Konrad\Desktop\Magisterka\Kod\2024-mgr-sluzba-cywilna\Python\Code\downloaded_file.csv
100%|██████████| 176k/176k [00:00<00:00, 2.78MB/s]
Downloading...
From: https://drive.google.com/uc?id=1YMX

In [44]:
# Removing unnecessary columns.

unique_columns_to_delete = [
    ["Transakcje rynkowe", "Powierzchnia użytkowa lokali mieszkalnych"],
    ["Mirgacje", "Wiek"],
    ["Ogółem"],
    ["Wskaźniki"],
    ["Wyszczególnienie"]
]

for i in range(len(bdl_dfs_list)):
    try:
        bdl_dfs_list[i].drop(["Nazwa", "Jednostka miary", "Atrybut"], axis=1, inplace=True)
        bdl_dfs_list[i] = bdl_dfs_list[i].iloc[:, :-1] # the last column has a different name in every df and is always unnecessary.
    except:
        pass

    # Columns unique for single dataframes.
    for columns in unique_columns_to_delete:
        try:
            bdl_dfs_list[i].drop(columns, axis=1, inplace=True)
        except:
            pass

In [45]:
cena_mieszkan = bdl_dfs_list[0].copy()
mediana_wieku = bdl_dfs_list[1].copy()
saldo_migracji = bdl_dfs_list[2].copy()
wsp_feminizacji = bdl_dfs_list[3].copy()
wydatki_powiat = bdl_dfs_list[4].copy()
wynagrodzenia = bdl_dfs_list[5].copy()

### Merging main with BDL

In [47]:
# Importing main data.
file_id = "1IlyXO5rXkLi28Z-o-WWCjaFa_rLDrKls"
url = f"https://drive.google.com/uc?id={file_id}"
file_extension = "downloaded_file.csv"
gdown.download(url, file_extension, quiet=False)
main_df = pd.read_csv(file_extension, sep=",")
main_df.head()

Downloading...
From (original): https://drive.google.com/uc?id=1IlyXO5rXkLi28Z-o-WWCjaFa_rLDrKls
From (redirected): https://drive.google.com/uc?id=1IlyXO5rXkLi28Z-o-WWCjaFa_rLDrKls&confirm=t&uuid=d0c00514-c911-4f36-8415-8757acebd195
To: C:\Users\Konrad\Desktop\Magisterka\Kod\2024-mgr-sluzba-cywilna\Python\Code\downloaded_file.csv
100%|██████████| 253M/253M [00:04<00:00, 61.4MB/s] 


,ad_id,result,job_field,position_category,position,education_level,work_time,vacancies,city,institution,...,year_announced,promocja,kandydaci,salary_for_position,offer_salary,women_percentage,fluctuation,employment_contract,voivodeship,district
0,130994,1,vet,stanowiska samodzielne,inspektor weterynaryjny,3,0.5,1,Olesno,Powiatowy Inspektorat Weterynarii w Oleśnie,...,2023,1.0,1.0,NaN,nie podano wynagrodzenia,0.666667,0.064683,1.19,16,8
1,130970,0,tech/construction,stanowiska samodzielne,starszy specjalista,3,1.0,1,Bydgoszcz,Wojewódzki Inspektorat Nadzoru Budowlanego w B...,...,2023,0.0,4.0,NaN,nie podano wynagrodzenia,0.580645,0.036010,0.50,4,61
2,130898,1,vet,stanowiska samodzielne,asystent,2,1.0,1,Chrzanów,Powiatowy Inspektorat Weterynarii w Chrzanowie,...,2023,0.0,18.0,NaN,"4400,00 zł brutto",0.833333,0.101215,NaN,12,3
3,130919,0,uniformed services,stanowiska samodzielne,specjalista,4,1.0,1,Warszawa,Komenda Stołeczna Policji w Warszawie,...,2023,NaN,NaN,NaN,"4940,34 zł brutto",NaN,NaN,NaN,14,65
4,130837,1,vet,stanowiska samodzielne,inspektor weterynaryjny,3,1.0,1,Świdnica,Powiatowy Inspektorat Weterynarii w Świdnicy,...,2023,1.0,2.0,10268.888889,nie podano wynagrodzenia,0.833333,0.000000,3.00,2,19
